In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import folium as fl

In [2]:
#Importing data into notebook
df_events = pd.read_csv('events_data.csv')
df_gender = pd.read_csv('gender_age_train.csv')
df_brand = pd.read_csv('phone_brand_device_model.csv')

In [3]:
df_events.shape

(3252950, 7)

In [4]:
df_events.isnull().sum()

event_id       0
device_id    453
timestamp      0
longitude    423
latitude     423
city           0
state        377
dtype: int64

In [5]:
df_gender.isnull().sum()

device_id    0
gender       0
age          0
group        0
dtype: int64

In [6]:
df_brand.isnull().sum()

device_id       0
phone_brand     0
device_model    0
dtype: int64

In [7]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
event_id     float64
device_id    float64
timestamp    object
longitude    float64
latitude     float64
city         object
state        object
dtypes: float64(4), object(3)
memory usage: 173.7+ MB


In [8]:
df_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
device_id    74645 non-null int64
gender       74645 non-null object
age          74645 non-null int64
group        74645 non-null object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [9]:
df_brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87726 entries, 0 to 87725
Data columns (total 3 columns):
device_id       87726 non-null int64
phone_brand     87726 non-null object
device_model    87726 non-null object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [ ]:
profile = pandas_profiling.ProfileReport(df_events)
profile.to_file('events_profile.html')

In [ ]:
profile2 = pandas_profiling.ProfileReport(df_gender)
profile2.to_file('gender_profile.html')

In [ ]:
profile3 = pandas_profiling.ProfileReport(df_brand)
profile3.to_file('brand_profile.html')

In [10]:
missing_state_city = df_events[df_events['state'].isnull()]['city'].unique()
missing_state_city

array(['Delhi', 'Pune', 'Chennai', 'Visakhapatnam', 'Indore', 'Jaipur',
       'Arambagh', 'Channapatna', 'Gangarampur'], dtype=object)

In [11]:
missing_state = []
for city in missing_state_city:
    missing_state.append(df_events.loc[df_events['city'] == city]['state'].unique()[0])
missing_state

['Delhi',
 'Maharashtra',
 'TamilNadu',
 'AndhraPradesh',
 'MadhyaPradesh',
 'Rajasthan',
 'WestBengal',
 'Karnataka',
 'WestBengal']

In [12]:
# replacing the missing state based on city's other state value
for i in range(0, len(missing_state)):
    cond = (df_events['city'] == missing_state_city[i])
    df_events.loc[cond, 'state'] = df_events.loc[cond, 'state'].fillna(missing_state[i])
df_events.state.isnull().sum()

0

In [51]:
# Filtering out the required state records
df = df_events[df_events['state'].isin(['Rajasthan', 'UttarPradesh', 'Haryana', 'Jharkhand', 'Orissa', 'Telangana', 'Assam'])]
df.head()

,event_id,device_id,timestamp,longitude,latitude,city,state
2458997,616897.0,4.112233e+18,2016-05-06 23:07:41,78.566758,17.481422,Hyderabad,Telangana
2458998,2699320.0,4.112233e+18,2016-05-04 12:34:38,78.566758,17.481422,Hyderabad,Telangana
2458999,3232372.0,4.112233e+18,2016-05-04 11:08:31,78.566758,17.481422,Hyderabad,Telangana
2459000,617054.0,8.845955e+18,2016-05-07 23:34:30,78.493221,17.476113,Hyderabad,Telangana
2459001,1440290.0,8.845955e+18,2016-05-07 23:31:37,78.493221,17.476113,Hyderabad,Telangana


In [52]:
df.shape

(370982, 7)

In [53]:
df.isnull().sum()

event_id      0
device_id    81
timestamp     0
longitude    66
latitude     66
city          0
state         0
dtype: int64

In [54]:
missing_lat = df[df['latitude'].isnull()][['event_id', 'city']]
missing_long = df[df['longitude'].isnull()][['event_id', 'city']]

In [55]:
print(missing_lat.shape)
print(missing_long.shape)

(66, 2)
(66, 2)


In [56]:
df[df['city'] == 'Jaipur']['latitude'].mean()

26.974426361189895

In [57]:
for i in range(0, 66):
    df.loc[df['event_id'] == missing_lat.iloc[i, 0], 'latitude'] = df[df['city'] == missing_lat.iloc[i, 1]]['latitude'].mean()
    df.loc[df['event_id'] == missing_long.iloc[i, 0], 'longitude'] = df[df['city'] == missing_long.iloc[i, 1]]['longitude'].mean()
    

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [58]:
df.dropna(inplace=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
df.isnull().sum()

event_id     0
device_id    0
timestamp    0
longitude    0
latitude     0
city         0
state        0
dtype: int64

In [61]:
df.shape

(370901, 7)

In [62]:
df_brand.shape

(87726, 3)

In [63]:
# Replacing brand names from other language to english
brand_name_og = ['华为','小米','三星','魅族','酷派','乐视','联想 ']
brand_name_en = ['Huawei','Xiaomi','Samsung','Meizu','Coolpad','LeEco','Lenovo']
for i in range(0, 7):
    df_brand['phone_brand'] = df_brand['phone_brand'].replace(brand_name_og[i], brand_name_en[i])

In [43]:
# df['device_id'] = df['device_id'].apply(lambda x: '{:.2f}'.format(x))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
df.to_csv('final_data.csv', index=False)

In [47]:
df.drop(labels='city', axis=1).to_csv('final_data.csv', index=False)

In [64]:
df_temp = pd.merge(df_gender, df_brand, how='left', on='device_id')
df_temp.head()

,device_id,gender,age,group,phone_brand,device_model
0,-8076087639492063270,M,35,M32-38,Xiaomi,MI 2
1,-2897161552818060146,M,35,M32-38,Xiaomi,MI 2
2,-8260683887967679142,M,35,M32-38,Xiaomi,MI 2
3,-4938849341048082022,M,30,M29-31,Samsung,Galaxy S4
4,245133531816851882,M,30,M29-31,SUGAR,时尚手机


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 370901 entries, 2458997 to 2829978
Data columns (total 7 columns):
event_id     370901 non-null float64
device_id    370901 non-null float64
timestamp    370901 non-null object
longitude    370901 non-null float64
latitude     370901 non-null float64
city         370901 non-null object
state        370901 non-null object
dtypes: float64(4), object(3)
memory usage: 22.6+ MB


In [69]:
df['device_id'] = df['device_id'].astype(int)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
merged_df = pd.merge(df, df_temp, how='left', on='device_id')
merged_df.shape

(370901, 12)

In [71]:
merged_df.head()

,event_id,device_id,timestamp,longitude,latitude,city,state,gender,age,group,phone_brand,device_model
0,616897.0,4112232520748712960,2016-05-06 23:07:41,78.566758,17.481422,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN
1,2699320.0,4112232520748712960,2016-05-04 12:34:38,78.566758,17.481422,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN
2,3232372.0,4112232520748712960,2016-05-04 11:08:31,78.566758,17.481422,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN
3,617054.0,8845954542270213120,2016-05-07 23:34:30,78.493221,17.476113,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN
4,1440290.0,8845954542270213120,2016-05-07 23:31:37,78.493221,17.476113,Hyderabad,Telangana,NaN,NaN,NaN,NaN,NaN


In [72]:
merged_df.isnull().sum()

event_id             0
device_id            0
timestamp            0
longitude            0
latitude             0
city                 0
state                0
gender          369272
age             369272
group           369272
phone_brand     369272
device_model    369272
dtype: int64